In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from typing import TypedDict, Annotated

from langgraph.graph import StateGraph, add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint import MemorySaver


from langchain_openai import ChatOpenAI
from langchain_community.tools import TavilySearchResults

In [5]:
memory = MemorySaver()

tool = TavilySearchResults(max_results=2)
tools = [tool]
tool_node = ToolNode(tools)

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

In [7]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state:State):
    result = llm_with_tools.invoke(state["messages"])
    return {"messages": [result]}

In [9]:
graph_builder = StateGraph(State)


graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)

graph_builder.set_entry_point("chatbot")

graph = graph_builder.compile(
    checkpointer=memory,
    interrupt_before=["tools"]
)

In [12]:
# 웹검색이 필요하다. toolnode가 필요한데, 실행이 되지 않는다.
user_input = "LangGraph가 뭐야?"
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)

for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()


================================ Human Message =================================

LangGraph가 뭐야?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_mhU2u2KU4GDL2MIwjATn73Kj)
 Call ID: call_mhU2u2KU4GDL2MIwjATn73Kj
  Args:
    query: LangGraph


In [13]:
# 웹검색이 필요가 없다.
user_input = "안녕?"
config = {"configurable": {"thread_id": "2"}}

events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)

for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()


================================ Human Message =================================

안녕?
================================== Ai Message ==================================

안녕하세요! 어떻게 도와드릴까요?
